# Каталоги Респонсум
Пример ноутбука для работы с каталогам Responsum

### Зададим тип установки

In [1]:
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop-pre'

## Инициализация

Первым шагом любого ноутбука для построения отчетов необходимо загрузить библиотеки, которые нам помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter 

In [2]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc

# создадим объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)

print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Объектов в media-каталоге: 4716


# Демографические переменные

### Все демографические переменные
Получим список всех демографических переменных

In [3]:
demo_cat = rcats.get_demo()
demo_cat

,varId,varName,varTitle,categories,from,to,catNum,catTitle
0,106,SEX,Пол,"[{'varId': 106, 'catNum': 1, 'title': 'Мужчины...",None,None,1,Мужчины
1,106,SEX,Пол,"[{'varId': 106, 'catNum': 1, 'title': 'Мужчины...",None,None,2,Женщины
2,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",None,None,1,Москва
3,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",None,None,2,Санкт-Петербург
4,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",None,None,27,Казань
...,...,...,...,...,...,...,...,...
93,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",None,None,2,Санкт-Петербург
94,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",None,None,27,Казань
95,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",None,None,47,Екатеринбург
96,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",None,None,64,Новосибирск


Экспорт каталога в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [4]:
writer = pd.ExcelWriter('demo_cat.xlsx')
demo_cat.to_excel(writer, 'demo_cat', index=False)
writer.save()

Экспорт каталога в csv. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [5]:
demo_cat.to_csv('demo_cat.csv', encoding = 'utf-16')

### Поиск по демографическим переменным

Найдем переменные, которые содержат текст "Численность"

In [6]:
rcats.get_demo(find_text='численность')

,varId,varName,varTitle,categories,from,to,catNum,catTitle
0,246,CITY_TYPE,Численность населения города до 02.02.2014,"[{'varId': 246, 'catNum': 1, 'title': '800 тыс...",None,None,1,800 тыс. чел. и более
1,246,CITY_TYPE,Численность населения города до 02.02.2014,"[{'varId': 246, 'catNum': 1, 'title': '800 тыс...",None,None,2,от 500 до 800 тыс.чел.
2,246,CITY_TYPE,Численность населения города до 02.02.2014,"[{'varId': 246, 'catNum': 1, 'title': '800 тыс...",None,None,3,от 100 до 500 тыс.чел.
3,246,CITY_TYPE,Численность населения города до 02.02.2014,"[{'varId': 246, 'catNum': 1, 'title': '800 тыс...",None,None,4,менее 100 тыс.чел.
4,350,CITY_TYPE2,Численность населения города,"[{'varId': 350, 'catNum': 1, 'title': '700 тыс...",None,None,1,700 тыс. чел. и более
5,350,CITY_TYPE2,Численность населения города,"[{'varId': 350, 'catNum': 1, 'title': '700 тыс...",None,None,2,от 400 до 700 тыс.чел.
6,350,CITY_TYPE2,Численность населения города,"[{'varId': 350, 'catNum': 1, 'title': '700 тыс...",None,None,3,от 100 до 400 тыс.чел.
7,350,CITY_TYPE2,Численность населения города,"[{'varId': 350, 'catNum': 1, 'title': '700 тыс...",None,None,4,менее 100 тыс.чел.


# Поиск в медиа каталоге

### Поиск идентификатора сайта "avito.ru"
Найдем все сайты в названия которых входит название avito.ru (поиск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```
<a id=’catalogs_site_avito’></a>

In [7]:
site_avito = rcats.find_media('avito.ru', branch = 'holding', find_in=['site'])
site_avito

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch
3500,15827,Avito.ru,15828,Avito.ru,23433,Avito.ru // Поиск по всем категориям,1933,Avito.ru // Во всех категориях (до 30.04.11),holding
3501,15827,Avito.ru,15828,Avito.ru,15829,Avito.ru // total,1777,Avito.ru // total,holding
3502,15827,Avito.ru,15828,Avito.ru,23440,Avito.ru // Личные вещи,5363,"Avito.ru // Личные вещи // Детская одежда, обу...",holding
3503,15827,Avito.ru,15828,Avito.ru,23439,Avito.ru // Недвижимость,5356,Avito.ru // Недвижимость // Земельные участки,holding
3504,15827,Avito.ru,15828,Avito.ru,23442,Avito.ru // Хобби и отдых,5392,Avito.ru // Хобби и отдых,holding
...,...,...,...,...,...,...,...,...,...
3639,15827,Avito.ru,15828,Avito.ru,23442,Avito.ru // Хобби и отдых,5388,Avito.ru // Хобби и отдых // Музыкальные инстр...,holding
3648,15827,Avito.ru,15828,Avito.ru,23438,Avito.ru // Животные,5353,Avito.ru // Животные,holding
3654,15827,Avito.ru,15828,Avito.ru,23443,Avito.ru // Для бизнеса,1925,Avito.ru // Для бизнеса (до 30.04.11),holding
3664,15827,Avito.ru,15828,Avito.ru,23438,Avito.ru // Животные,5347,Avito.ru // Животные // Корм для животных,holding


Из полученного результата видим, что нас интересует сайт с __ID = 15828__

Экспорт каталога в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [8]:
writer = pd.ExcelWriter('site_avito.xlsx')
site_avito.to_excel(writer, 'site_avito', index=False)
writer.save()

Экспорт каталога в csv. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [9]:
site_avito.to_csv('site_avito.csv', encoding = 'utf-16')

### Поиск идентификатора сайта "ivi.ru"
Найдем все сайты в названия которых входит название ivi.ru (пойск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```

<a id=’catalogs_site_ivi’></a>

In [10]:
site_ivi = rcats.find_media('ivi.ru', branch = 'holding', find_in=['site'])
site_ivi

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch
3677,15868,Ivi.ru,15869,Ivi.ru,15870,Ivi.ru // Другие страницы,1793,Ivi.ru // Другие страницы,holding
3678,15868,Ivi.ru,15869,Ivi.ru,41592,Ivi.ru // Кино,14791,Ivi.ru // Кино // Военные,holding
3679,15868,Ivi.ru,15869,Ivi.ru,17593,Ivi-music.ru // Другие страницы,2546,Ivi-music.ru // Другие страницы,holding
3680,15868,Ivi.ru,15869,Ivi.ru,41592,Ivi.ru // Кино,15242,Ivi.ru // Кино // Фантастика,holding
3681,15868,Ivi.ru,15869,Ivi.ru,41592,Ivi.ru // Кино,43589,Ivi.ru // Кино // Короткометражки,holding
...,...,...,...,...,...,...,...,...,...
3829,15868,Ivi.ru,15869,Ivi.ru,41612,Ivi.ru // Мультфильмы,362092,Ivi.ru // Мультфильмы // Приключения,holding
3830,15868,Ivi.ru,15869,Ivi.ru,41592,Ivi.ru // Кино,43554,Ivi.ru // Кино // Романтические,holding
3837,15868,Ivi.ru,15869,Ivi.ru,41616,Ivi.ru // Программы,209923,Ivi.ru // Программы // Взрослые,holding
3850,15868,Ivi.ru,15869,Ivi.ru,41612,Ivi.ru // Мультфильмы,210391,Ivi.ru // Мультфильмы // Развивающие,holding


Из полученного результата видим, что нас интересует сайт с __ID = 15869__

Экспорт каталога в Excel

In [11]:
writer = pd.ExcelWriter('site_ivi.xlsx')
site_ivi.to_excel(writer, 'site_ivi', index=False)
writer.save()

Экспорт каталога в csv

In [12]:
site_ivi.to_csv('site_ivi.csv', encoding = 'utf-16')

### Поиск идентификатора сайта "kommerasnt.ru"
Найдем все сайты в названия которых входит название kommerasnt.ru (пойск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```

In [13]:
site_kommersant = rcats.find_media('kommersant.ru', branch = 'holding', find_in=['site'])
site_kommersant.head()

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch
2802,12,ИД КоммерсантЪ,118,Kommersant.ru,1113,Kommersant.ru // Главная страница,113,Kommersant.ru // Главная страница,holding
2804,12,ИД КоммерсантЪ,118,Kommersant.ru,1110,Kommersant.ru // Ростов,110,Kommersant.ru // Ростов,holding
2806,12,ИД КоммерсантЪ,118,Kommersant.ru,13032,Kommersant.ru // Газета,674,Kommersant.ru // Газета,holding
2808,12,ИД КоммерсантЪ,118,Kommersant.ru,13034,Kommersant.ru // Видео,673,Kommersant.ru // Видео,holding
2810,12,ИД КоммерсантЪ,118,Kommersant.ru,1117,Kommersant.ru // Y-docs,117,Kommersant.ru // Y-docs,holding


Из полученного результата видим, что нас интересует сайт с __ID = 118__

### Поиск идентификатора сайта "Vk.com"
Найдем все сайты в названия которых входит название Vk.com (пойск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```

In [14]:
site_vk = rcats.find_media('Vk.com', branch = 'holding', find_in='site')
site_vk

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch
2326,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,116821,Vk.com // Mobile app // Вконтакте,holding
2327,10,Mail.Ru Group,16571,Vk.com,16572,Vk.com // total,2099,Vk.com // total,holding
2328,10,Mail.Ru Group,16571,Vk.com,73403,hidden_16571,31178,hidden_16571,holding
2329,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,226612,Vk.com // Mobile app // Snapster,holding
2330,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,411690,Vk.com // Mobile app // Vinci,holding
2331,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,478764,Vk.com // Mobile app // VK Live,holding
2333,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,584615,Vk.com // Mobile app // VK Admin,holding
2334,10,Mail.Ru Group,16571,Vk.com,214679,mobile_hidden_16571,149451,mobile_hidden_16571,holding


Из полученного результата видим, что нас интересует сайт с __ID = 16571__

### Поиск идентификатора сайта "Odnoklassniki.ru"
Найдем все сайты в названия которых входит название Odnoklassniki.ru (пойск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```

In [15]:
holding_ok = rcats.find_media('Odnoklassniki.ru', branch = 'holding', find_in='site')
holding_ok

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch
2251,10,Mail.Ru Group,12808,Odnoklassniki.ru,12864,Odnoklassniki.ru // Знакомства,612,Odnoklassniki.ru // Знакомства,holding
2252,10,Mail.Ru Group,12808,Odnoklassniki.ru,12809,Odnoklassniki.ru // total,588,Odnoklassniki.ru // total,holding
2253,10,Mail.Ru Group,12808,Odnoklassniki.ru,12862,Odnoklassniki.ru // Работа,611,Odnoklassniki.ru // Работа,holding
2255,10,Mail.Ru Group,12808,Odnoklassniki.ru,207909,Odnoklassniki.ru // Mobile app,411753,Odnoklassniki.ru // Mobile app // ОК Сообщения,holding
2256,10,Mail.Ru Group,12808,Odnoklassniki.ru,72463,hidden_12808,30708,hidden_12808,holding
2257,10,Mail.Ru Group,12808,Odnoklassniki.ru,207909,Odnoklassniki.ru // Mobile app,116763,Odnoklassniki.ru // Mobile app // Odnoklassniki,holding
2258,10,Mail.Ru Group,12808,Odnoklassniki.ru,207909,Odnoklassniki.ru // Mobile app,361670,Odnoklassniki.ru // Mobile app // Мастерская п...,holding
2259,10,Mail.Ru Group,12808,Odnoklassniki.ru,207909,Odnoklassniki.ru // Mobile app,155426,Odnoklassniki.ru // Mobile app // Модератор,holding
2260,10,Mail.Ru Group,12808,Odnoklassniki.ru,214212,mobile_hidden_12808,148984,mobile_hidden_12808,holding
2266,10,Mail.Ru Group,12808,Odnoklassniki.ru,207909,Odnoklassniki.ru // Mobile app,411663,Odnoklassniki.ru // Mobile app // OK Live,holding


Из полученного результата видим, что нас интересует сайт с __ID = 12808__

### Поиск идентификатора холдинга "Google"
Найдем все холдинги в названия которых входит название Google Projects (пойск выполняется без учета регистра).

Искать будем только по ветке "holding", для этого зададим 

```
branch = 'holding'
find_in = ['holding']
```

In [16]:
holding_ok = rcats.find_media('google', branch='holding', find_in=['holding'])
holding_ok

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch
450,88149,Google Sites,88150,Google (ru+com),88151,hidden_88150,38458,hidden_88150,holding
451,88149,Google Sites,88150,Google (ru+com),88153,Google (ru+com),38459,Google (ru+com),holding
452,88149,Google Sites,88150,Google (ru+com),208767,Google (ru+com) // Mobile app,123340,Google (ru+com) // Mobile app,holding
453,88149,Google Sites,88150,Google (ru+com),215152,mobile_hidden_88150,149924,mobile_hidden_88150,holding
458,88149,Google Sites,341364,Google // Play,341365,hidden_341364,388606,hidden_341364,holding
459,88149,Google Sites,341364,Google // Play,341366,mobile_hidden_341364,388607,mobile_hidden_341364,holding
460,88149,Google Sites,341364,Google // Play,341367,Google // Play // Mobile app,388608,Google // Play // Mobile app,holding
461,88149,Google Sites,341364,Google // Play,341401,Google // Play,388609,Google // Play,holding
466,88149,Google Sites,88155,Youtube.com,88158,Youtube.com // total,38461,Youtube.com // total,holding
467,88149,Google Sites,88155,Youtube.com,88156,hidden_88155,38460,hidden_88155,holding


Из полученного результата видим, что нас интересует холдинг с __ID = 88149__

### Поиск идентификатора для brand
Также можно найти идентификатор нужного бренда, например, для отчета post campaign

Искать будем только в названиях проектов ветки adagency (они же Бренды для рекламной ветки)

```
branch = 'agency'
find_in = ['site']


In [17]:
#шаблон запроса на поиск идентификатора бренда
brand_x = rcats.find_media(find_text='VI', branch = 'agency', find_in=['site'])
brand_x

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch


### Пример поиска по холдингу "Mail.ru"


In [18]:
# 16571 - Vk.com
rcats.find_media('16571', find_in=['site'])

,holding_id,holding_title,site_id,site_title,section_id,section_title,subsection_id,subsection_title,branch
2326,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,116821,Vk.com // Mobile app // Вконтакте,holding
2327,10,Mail.Ru Group,16571,Vk.com,16572,Vk.com // total,2099,Vk.com // total,holding
2328,10,Mail.Ru Group,16571,Vk.com,73403,hidden_16571,31178,hidden_16571,holding
2329,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,226612,Vk.com // Mobile app // Snapster,holding
2330,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,411690,Vk.com // Mobile app // Vinci,holding
2331,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,478764,Vk.com // Mobile app // VK Live,holding
2333,10,Mail.Ru Group,16571,Vk.com,207918,Vk.com // Mobile app,584615,Vk.com // Mobile app // VK Admin,holding
2334,10,Mail.Ru Group,16571,Vk.com,214679,mobile_hidden_16571,149451,mobile_hidden_16571,holding
